<a href="https://colab.research.google.com/github/kocielnik/rule_one_stocks/blob/main/rule_one_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
%%file deals.py

"""
Sue is an aspiring Rule One Investor.

She wants to check stock prices daily to see if any Rule One deals are available
for her.

For simplicity and robustness of examples, the `Ticker` class in these examples
is replaced with MockTicker.

When Sue uses this program, she always omits the part `Ticker=MockTicker`.


One day, Sue wants to check the price of a single stock.

>>> company = Company(symbol="tsm", sticker_price=11)
>>> get_deal(company, Ticker=MockTicker)
Deal(symbol='tsm', sticker_price=11, price=98, percent_of_sticker=891)


On another day, Sue has several companies she wants to check the price of.

To the same method `get_deal`, she can pass a list of companies the same way
she passed a single company.

>>> companies = [
...  Company(symbol="msft", sticker_price=118),
...  Company(symbol="tsm", sticker_price=11)
... ]
>>> deals = get_deal(companies, Ticker=MockTicker)
>>> deals[0]
Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
>>> deals[1]
Deal(symbol='tsm', sticker_price=11, price=98, percent_of_sticker=891)


Sue wants the results to be sorted in the order of the best deal available.

Best deal means the largest Margin of Safety (MOS).

If companies are not sorted in the order of the best deal, the results *should*
be.

Note the order of input companies is reversed with respect to the previous example.

>>> companies = [
...  Company(symbol="tsm", sticker_price=11),
...  Company(symbol="msft", sticker_price=118)
... ]
>>> deals = get_deal(companies, Ticker=MockTicker)
>>> deals[0]
Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
"""

from types import SimpleNamespace
from yfinance import Ticker


class Company():
    def __init__(self, symbol, sticker_price):
        self.symbol = symbol
        self.sticker_price = sticker_price


class Deal(SimpleNamespace):

    """
    Sue wants to print an instance of this class and be able to copy-paste
    it into another place, or save it for later pasting.

    >>> Deal(symbol='msft', sticker_price=1, price=1)
    Deal(symbol='msft', sticker_price=1, price=1)


    Sue wants to be able to compare instances of `Deal` to check for equality,
    in order to remove duplicates from lists if any are found.

    She does not remember, *why* exactly she wanted this feature. Still, she
    insists it should be available.

    >>> deal_1 = Deal(symbol='msft', sticker_price=1, price=1)
    >>> deal_2 = Deal(symbol='msft', sticker_price=1, price=1)
    >>> deal_1 == deal_2
    True
    """

    def __eq__(self, other):
        if repr(other) != repr(self):
            return False
        return True


class MockTicker:
    def __init__(self, symbol):
        price = 352.5 if symbol == "msft" else 98

        self.info = {
            "currentPrice": price
        }


def get_price(symbol, Ticker=Ticker):

    """
    >>> get_price("msft", Ticker=MockTicker)
    352.5
    >>> get_price([])
    {}
    >>> get_price(["msft"], Ticker=MockTicker)
    {'msft': 352.5}
    """

    if type(symbol) == str:
        price = Ticker(symbol).info["currentPrice"]
        return price

    prices = {
        the_symbol: get_price(the_symbol, Ticker=Ticker)
        for the_symbol in symbol
    }
    return prices

def get_percent_of_sticker(price, sticker_price):
    return int(100 * round(price/sticker_price, 2))

def get_deal(company, Ticker=Ticker):

    """
    >>> company = Company(symbol="msft", sticker_price=118)
    >>> deal = get_deal(company, Ticker=MockTicker)
    >>> deal
    Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
    """

    if isinstance(company, list):
        return get_deals(company, Ticker=Ticker)

    price = get_price(company.symbol, Ticker=Ticker)
    return Deal(
        symbol=company.symbol,
        sticker_price=company.sticker_price,
        price=price,
        percent_of_sticker=get_percent_of_sticker(price, company.sticker_price)
    )

def get_deals(companies, Ticker=Ticker):

    """
    >>> companies = [
    ...   Company("msft", 118),
    ...   Company("tsm", 22)
    ... ]
    >>> deals = get_deals(companies, Ticker=MockTicker)
    >>> deals[0]
    Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
    """

    sticker_prices = {
        company.symbol: company.sticker_price
        for company in companies
    }

    deals = []
    for symbol in sticker_prices.keys():
        price = get_price(symbol, Ticker=Ticker)
        sticker_price=sticker_prices[symbol]
        deals.append(
            Deal(symbol=symbol,
                 sticker_price=sticker_price,
                 price=price,
                 percent_of_sticker=get_percent_of_sticker(price, sticker_price)
            )
        )
        
    sorted_deals = sorted(deals, key=lambda x: x.percent_of_sticker)
    
    return sorted_deals

Overwriting deals.py


In [99]:
!pytest --doctest-modules deals.py

============================= test session starts ==============================
platform linux -- Python 3.10.11, pytest-7.2.2, pluggy-1.0.0
rootdir: /content
plugins: anyio-3.6.2
collected 5 items                                                              

deals.py .....                                                           [100%]

============================== 5 passed in 0.54s ===============================
